## 🧠 Key Concept: What Are Embeddings?

Before we dive in, let's understand the magic behind RAG: **Embeddings**.

### The Problem
Computers don't understand text. They only understand numbers.

### The Solution: Embeddings
**Embeddings convert text into a list of numbers (a vector)** that captures the *meaning* of the text.

```
"I love pizza" → [0.12, -0.45, 0.89, 0.33, ..., 0.21]  (1536 numbers for OpenAI)
"Pizza is great" → [0.11, -0.44, 0.87, 0.35, ..., 0.19]  (very similar!)
"I hate Mondays" → [-0.55, 0.22, 0.11, -0.67, ..., 0.45]  (very different!)
```

### Why This Matters
- Similar meanings → Similar vectors → Close in "vector space"
- We can find relevant documents by finding vectors closest to the question's vector
- This is called **semantic search** (search by meaning, not just keywords)

---

## 🔧 Setup

Let's load our environment and import the necessary libraries.

In [ ]:
import os
from dotenv import load_dotenv

# Modern LangChain imports (2025 standard - split packages)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Load API key from .env file
load_dotenv()

# Verify API key
if os.getenv("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY loaded successfully!")
else:
    print("❌ ERROR: OPENAI_API_KEY not found. Please check your .env file.")

print("📦 All libraries imported successfully!")

---

## Step 1: 📄 Load Data

First, we need to get our documents into the system. LangChain provides **Document Loaders** for various file types:
- `TextLoader` → Plain text files
- `PyPDFLoader` → PDF files
- `CSVLoader` → CSV files
- And many more!

For this example, we'll use a simple text file with some "secret" information.

In [ ]:
# ===========================================
# STEP 1: LOAD DATA
# ===========================================

# Load our sample text file
# This file contains information that the base LLM doesn't know!
loader = TextLoader("sample.txt")
documents = loader.load()

print("📄 LOADED DOCUMENTS")
print("=" * 50)
print(f"Number of documents: {len(documents)}")
print(f"\nDocument content:")
print("-" * 50)
print(documents[0].page_content)
print("-" * 50)
print(f"\nMetadata: {documents[0].metadata}")

---

## Step 2: ✂️ Split Data into Chunks

### Why Split?

1. **Context Window Limits**: LLMs can only process so much text at once
2. **Relevance**: Smaller chunks = more precise retrieval
3. **Cost**: Sending entire documents is expensive

### The RecursiveCharacterTextSplitter

This splitter tries to keep related content together by:
1. First trying to split on paragraphs (`\n\n`)
2. Then sentences (`\n`)
3. Then spaces
4. Finally, characters

### Why Overlap Matters! 🔑

```
Without overlap:  [Chunk 1: "The vault code is"] [Chunk 2: "9988. The manager"]
                   ↑ This chunk misses the code!   ↑ This chunk misses context!

With overlap:     [Chunk 1: "The vault code is 9988."] [Chunk 2: "code is 9988. The manager"]
                   ↑ Complete information!           ↑ Overlapping context preserved!
```

**Overlap ensures that important information at chunk boundaries isn't lost!**

In [ ]:
# ===========================================
# STEP 2: SPLIT DATA INTO CHUNKS
# ===========================================

# Create a text splitter
# - chunk_size: Maximum characters per chunk
# - chunk_overlap: Characters to repeat between chunks (preserves context!)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,      # Small chunks for demonstration
    chunk_overlap=20,    # 20 characters overlap between chunks
    length_function=len,
    separators=["\n\n", "\n", " ", ""]  # Try these separators in order
)

# Split the documents
chunks = text_splitter.split_documents(documents)

print("✂️ SPLIT INTO CHUNKS")
print("=" * 50)
print(f"Original documents: {len(documents)}")
print(f"After splitting: {len(chunks)} chunks")
print(f"\nChunk size: 100 characters")
print(f"Overlap: 20 characters")
print("\n" + "=" * 50)
print("📦 INDIVIDUAL CHUNKS:")
print("=" * 50)

for i, chunk in enumerate(chunks, 1):
    print(f"\n🔹 Chunk {i} ({len(chunk.page_content)} chars):")
    print(f"   '{chunk.page_content}'")

---

## Step 3: 🔢 Embed & Store

Now we convert our text chunks into **embeddings** (vectors) and store them in a **vector database**.

### What Happens Here:

1. **OpenAIEmbeddings**: Calls OpenAI's API to convert each chunk into a 1536-dimensional vector
2. **Chroma**: A lightweight vector database that stores these vectors and allows fast similarity search

### The Magic
```
"The vault code is 9988" → [0.12, -0.45, 0.89, ..., 0.21] (1536 numbers)
"What is the vault code?" → [0.11, -0.44, 0.88, ..., 0.20] (similar numbers!)
```

When we search, we find chunks with vectors most similar to our question's vector!

In [ ]:
# ===========================================
# STEP 3: EMBED & STORE
# ===========================================

# Initialize the embedding model
# This converts text → vectors (lists of numbers)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"  # Cost-effective embedding model
)

print("🔢 CREATING EMBEDDINGS & VECTOR STORE")
print("=" * 50)

# Create the vector store from our chunks
# This does several things:
# 1. Sends each chunk to OpenAI to get its embedding
# 2. Stores the chunks and their embeddings in Chroma
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="rag_demo"  # Name for this collection
)

print("✅ Vector store created!")
print(f"📊 Stored {len(chunks)} chunks with embeddings")

In [ ]:
# Let's see what an embedding looks like!
sample_text = "What is the vault code?"
sample_embedding = embeddings.embed_query(sample_text)

print("🔍 EMBEDDING EXAMPLE")
print("=" * 50)
print(f"Text: '{sample_text}'")
print(f"\nEmbedding dimensions: {len(sample_embedding)}")
print(f"First 10 values: {sample_embedding[:10]}")
print(f"\n💡 Each text becomes a {len(sample_embedding)}-dimensional vector!")

---

## Step 4: 🎯 Retrieve Relevant Chunks

Now let's test the retrieval! When we ask a question:
1. The question is converted to an embedding
2. We find the chunks with the most similar embeddings
3. Those chunks are returned as "context"

In [ ]:
# ===========================================
# STEP 4: RETRIEVE RELEVANT CHUNKS
# ===========================================

# Create a retriever from the vector store
# k=2 means: return the 2 most relevant chunks
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)

print("🎯 TESTING RETRIEVAL")
print("=" * 50)

# Test with a question
test_question = "What is the vault code?"
retrieved_docs = retriever.invoke(test_question)

print(f"Question: '{test_question}'")
print(f"\nRetrieved {len(retrieved_docs)} relevant chunks:\n")

for i, doc in enumerate(retrieved_docs, 1):
    print(f"📄 Chunk {i}:")
    print(f"   '{doc.page_content}'")
    print()

In [ ]:
# Let's test another question
test_question2 = "Who is the manager?"
retrieved_docs2 = retriever.invoke(test_question2)

print(f"Question: '{test_question2}'")
print(f"\nRetrieved {len(retrieved_docs2)} relevant chunks:\n")

for i, doc in enumerate(retrieved_docs2, 1):
    print(f"📄 Chunk {i}:")
    print(f"   '{doc.page_content}'")
    print()

---

## Step 5: 🤖 Generate Answer with RAG Chain

Now we put it all together! We'll create a **RAG Chain** using LangChain Expression Language (LCEL).

### The Chain Flow:

```
Question → [Retriever finds context] → [Prompt combines context + question] → [LLM generates answer]
```

### LCEL Syntax:
```python
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)
```

This reads as: "Take the input, use it to get context from retriever, pass the question through, then send to prompt, then to model, then parse the output."

In [ ]:
# ===========================================
# STEP 5: BUILD THE RAG CHAIN
# ===========================================

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create the prompt template
# This tells the LLM how to use the retrieved context
prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Answer the question based ONLY on the following context.
If the answer is not in the context, say "I don't have that information."

Context:
{context}

Question: {question}

Answer:
""")

# Helper function to format retrieved documents
def format_docs(docs):
    """Combine retrieved documents into a single string."""
    return "\n\n".join(doc.page_content for doc in docs)

# Build the RAG chain using LCEL (LangChain Expression Language)
rag_chain = (
    {
        "context": retriever | format_docs,  # Get docs and format them
        "question": RunnablePassthrough()     # Pass the question through unchanged
    }
    | prompt      # Fill in the prompt template
    | llm         # Send to the LLM
    | StrOutputParser()  # Extract the string response
)

print("✅ RAG Chain created!")
print("\n📊 Chain components:")
print("   1. Retriever → Finds relevant context")
print("   2. Prompt → Combines context + question")
print("   3. LLM → Generates answer")
print("   4. Parser → Extracts text response")

In [ ]:
# ===========================================
# TEST THE RAG CHAIN!
# ===========================================

print("🚀 RAG CHAIN IN ACTION!")
print("=" * 50)

# Question 1: The vault code
question1 = "What is the vault code?"
answer1 = rag_chain.invoke(question1)

print(f"❓ Question: {question1}")
print(f"✅ Answer: {answer1}")
print()

In [ ]:
# Question 2: The manager's name
question2 = "Who is the manager?"
answer2 = rag_chain.invoke(question2)

print(f"❓ Question: {question2}")
print(f"✅ Answer: {answer2}")
print()

In [ ]:
# Question 3: Something NOT in the document
question3 = "What is the company's revenue?"
answer3 = rag_chain.invoke(question3)

print(f"❓ Question: {question3}")
print(f"✅ Answer: {answer3}")
print()
print("💡 The model correctly says it doesn't have that information!")

In [ ]:
# Question 4: More complex question
question4 = "What are both the primary and backup vault codes?"
answer4 = rag_chain.invoke(question4)

print(f"❓ Question: {question4}")
print(f"✅ Answer: {answer4}")

---

## 🔬 Bonus: See the Full RAG Process

Let's create a verbose version that shows each step of the RAG process.

In [ ]:
def ask_with_details(question: str):
    """
    Ask a question and show the full RAG process.
    """
    print("🔍 FULL RAG PROCESS")
    print("=" * 60)
    
    # Step 1: Show the question
    print(f"\n📝 QUESTION: {question}")
    
    # Step 2: Retrieve relevant chunks
    print("\n🎯 STEP 1: RETRIEVAL")
    print("-" * 40)
    retrieved = retriever.invoke(question)
    for i, doc in enumerate(retrieved, 1):
        print(f"   Chunk {i}: '{doc.page_content[:60]}...'")
    
    # Step 3: Show formatted context
    context = format_docs(retrieved)
    print("\n📄 STEP 2: FORMATTED CONTEXT")
    print("-" * 40)
    print(f"   {context[:200]}...")
    
    # Step 4: Generate answer
    print("\n🤖 STEP 3: LLM GENERATION")
    print("-" * 40)
    answer = rag_chain.invoke(question)
    print(f"   {answer}")
    
    print("\n" + "=" * 60)
    return answer

# Test it!
ask_with_details("What is the backup code and when can I use it?")

---

## 📝 Summary

### The RAG Pipeline

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   LOAD      │ →  │   SPLIT     │ →  │   EMBED     │ →  │   STORE     │
│ Documents   │    │ Into chunks │    │ Text→Vector │    │ Vector DB   │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
                                                               ↓
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   ANSWER    │ ←  │  GENERATE   │ ←  │  RETRIEVE   │ ←  │  QUESTION   │
│  To user    │    │ With LLM    │    │ Similar     │    │ From user   │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
```

### Key Concepts

✅ **Embeddings**
- Convert text to vectors (lists of numbers)
- Similar meanings → Similar vectors
- Enables semantic search (by meaning, not keywords)

✅ **Chunking**
- Break documents into smaller pieces
- Overlap preserves context at boundaries
- Smaller chunks = more precise retrieval

✅ **Vector Store (Chroma)**
- Stores chunks + their embeddings
- Enables fast similarity search
- Returns most relevant chunks for a query

✅ **RAG Chain (LCEL)**
- Retriever finds context
- Prompt combines context + question
- LLM generates grounded answer

### Modern Imports (2025)

```python
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
```

---

### 🚀 Next Steps

Now you understand RAG! In the Projects module, you'll build complete applications like:
- Chat with your PDFs
- Q&A over your codebase
- Custom knowledge assistants

Happy building! 🎉

In [ ]:
# ===========================================
# CLEANUP (Optional)
# ===========================================

# If you want to clean up the in-memory vector store
# vectorstore.delete_collection()
# print("🧹 Vector store cleaned up!")

print("\n🎉 Congratulations! You've completed the RAG Basics module!")
print("You now know how to:")
print("   ✅ Load documents")
print("   ✅ Split them into chunks")
print("   ✅ Create embeddings")
print("   ✅ Store in a vector database")
print("   ✅ Retrieve relevant context")
print("   ✅ Generate answers with RAG")